In [1]:
from gym.wrappers import RescaleAction
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

from environments import ARESEAOneStep, ARESEASequential

## Sequential

## Onestep

In [4]:
run_name = "happy-snowflake-22"
log_dir = f"models/{run_name}"

def make_env():
    env = ARESEAOneStep(
        backend="simulation",
        random_incoming=True,
        random_initial=True,
        beam_parameter_method="direct"
    )
    env = RescaleAction(env, -1, 1)
    return env

env = DummyVecEnv([make_env])
env = VecNormalize.load(f"{log_dir}/vec_normalize.pkl", env)
env.training = False
env.norm_reward = False

model = PPO.load(f"{log_dir}/model")

In [19]:
observation = env.reset()
action, _ = model.predict(observation)
observation, reward, done, info = env.step(action)
observation, reward, done, info

(array([[ 0.48228568,  1.0815147 ,  0.76897246, -0.35417643,  0.9074537 ,
         -1.0188341 ,  0.3069467 ,  1.0955995 ,  0.5280139 ,  0.10682743,
         -0.58083767, -0.38411614,  1.1999596 ]], dtype=float32),
 array([6.904102], dtype=float32),
 array([ True]),
 [{'terminal_observation': array([ 0.15661132, -0.76719594,  0.9124457 , -0.70674366,  1.1307076 ,
           1.6147237 , -1.3965282 ,  0.543679  ,  0.0691342 ,  0.12852256,
          -0.75333804, -0.6109389 , -0.5976856 ], dtype=float32)}])

In [21]:
env.get_original_obs()

array([[ 8.3123465e+00,  1.8768345e+01,  1.3367793e+01, -6.1579561e-04,
         3.1429308e-03, -1.1821007e-03,  3.5710909e-04,  4.4264336e-04,
         3.4305317e-04,  1.4438507e-03, -1.3354638e-03,  1.7808332e-03,
         1.3824385e-02]], dtype=float32)